In [ ]:
def solve(self,save_every,tracer):
        kw_t="transport"
        node_discretization = pp.Upwind(kw_t)
        source_discretization = pp.ScalarSource(kw_t)
        mass_discretization = pp.MassMatrix(kw_t)
        
        node_discretization.discretize(self.g,self.data)
        source_discretization.discretize(self.g,self.data)
        mass_discretization.discretize(self.g,self.data)
        
        A_upwind,b_upwind=node_discretization.assemble_matrix_rhs(self.g,self.data)
        #_,b_source=source_discretization.assemble_matrix_rhs(self.g,self.data)
        A_mass,b_mass=mass_discretization.assemble_matrix_rhs(self.g,self.data)
        
        dt=self.data[pp.PARAMETERS]["transport"]["time_step"]
        decay=self.data[pp.PARAMETERS]["transport"]["lambda_lin_decay"]
        
        if self.data[pp.PARAMETERS]["transport"]["method"]=="Explicit":
            lhs =1/dt*A_mass
        else:
            lhs=1/dt*A_mass+A_upwind
            #rhs=b_upwind+b_source+b_mass
        
        rhs=b_upwind+b_mass
        IEsolver = sps.linalg.factorized(lhs)
        
        n_steps = int(np.round(self.data[pp.PARAMETERS]["transport"]["t_max"] / self.data[pp.PARAMETERS]["transport"]["time_step"]))
        
        # Exporter
        exporter = pp.Exporter(self.g, file_name="tracer",folder_name="solution")
        print("done")
        
        for i in range(n_steps):
            if np.isclose(i % save_every, 0):
                # Export existing solution (final export is taken care of below)
                exporter.write_vtu({"tracer":tracer}, time_step=int(i // save_every))
                print(tracer)
                if self.data[pp.PARAMETERS]["transport"]["method"]=="Explicit":
                    tracer = IEsolver((1/dt*A_mass-A_upwind-decay*A_mass)*tracer+rhs)
                else:
                    tracer = IEsolver((1/dt*A_mass-decay*A_mass) * tracer + rhs)
        
        exporter.write_vtu({"tracer":tracer}, time_step=(n_steps // save_every))
        print(tracer)
        time_steps = np.arange(0,self.data[pp.PARAMETERS]["transport"]["t_max"] + self.data[pp.PARAMETERS]["transport"]["time_step"], save_every * self.data[pp.PARAMETERS]["transport"]["time_step"])
        exporter.write_pvd(time_steps)        